In [1]:
import os
import json
from pathlib import Path

import sagemaker
import boto3
from dotenv import load_dotenv

ROOT = Path.cwd().parent.parent

REGION_NAME = "eu-central-1"
ROLE_NAME = "Sagemaker-ExecutionRole"

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/hmosousa/.config/sagemaker/config.yaml


## Auth

In [2]:
load_dotenv(ROOT / ".env")

os.environ["AWS_ACCESS_KEY_ID"] = os.getenv("AWS_ID")
os.environ["AWS_SECRET_ACCESS_KEY"] = os.getenv("AWS_KEY")
os.environ["AWS_DEFAULT_REGION"] = REGION_NAME

auth_args = {
    "aws_access_key_id": os.getenv("AWS_ID"),
    "aws_secret_access_key": os.getenv("AWS_KEY"),
    "region_name": REGION_NAME,
}

In [3]:
iam = boto3.client("iam", **auth_args)
role = iam.get_role(RoleName=ROLE_NAME)["Role"]["Arn"]

session = sagemaker.Session(boto3.Session(**auth_args))

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/hmosousa/.config/sagemaker/config.yaml


## Deploy

In [4]:
from sagemaker.huggingface import HuggingFaceModel, get_huggingface_llm_image_uri

hub = {
	'HF_MODEL_ID':'tiiuae/falcon-40b',
	'SM_NUM_GPUS': json.dumps(4)
}

# create Hugging Face Model Class
huggingface_model = HuggingFaceModel(
	image_uri=get_huggingface_llm_image_uri("huggingface",version="0.9.3"),
	env=hub,
	role=role, 
)

# deploy model to SageMaker Inference
predictor = huggingface_model.deploy(
	initial_instance_count=1,
	instance_type="ml.g5.12xlarge",
	container_startup_health_check_timeout=300,
)

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/hmosousa/.config/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/hmosousa/.config/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/hmosousa/.config/sagemaker/config.yaml


------------------*

UnexpectedStatusException: Error hosting endpoint huggingface-pytorch-tgi-inference-2023-09-15-17-24-33-841: Failed. Reason: The primary container for production variant AllTraffic did not pass the ping health check. Please check CloudWatch logs for this endpoint..

## Inference

In [ ]:
prompt = "To complete this sentence "

request = {
    "inputs": prompt,
    "parameters": {
        "do_sample": False,
        "temperature": 0,
        "max_new_tokens": 10,
    }  
}

response = predictor.predict(request)

answer = response[0]["generated_text"][len(prompt):]

## Clean up

In [ ]:
predictor.delete_model()
predictor.delete_endpoint()